In [1]:
import tensorflow as tf

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Input, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import os
from os import listdir

2024-07-13 12:05:51.815904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 12:05:51.815962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 12:05:51.873348: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-13 12:05:51.916165: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = "Data_Cancer/"
labels = listdir(data_dir)
labels

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [3]:
img_height = 224
img_width = 224
batch_size = 64

### Train, validation and Test 

In [4]:
def load_images(path):
    x = []
    y = []
    labels = os.listdir(path)
    for index, folder in enumerate(os.listdir(path)):
        for image in os.listdir(path+"/"+folder):
            image_path = path+"/"+folder+"/"+image
            img = cv2.imread(image_path)
            img = np.array(img)
            x.append(img)
            y.append(index)
    x = np.array(x)
    y = np.array(y)
    return x,y, labels

In [5]:
train_dataset = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "training",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

validation_dataset = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "validation",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.


2024-07-13 12:06:08.169957: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 12:06:08.371767: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 12:06:08.371839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 12:06:08.375012: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-13 12:06:08.375077: I external/local_xla/xla/stream_executor

Found 10015 files belonging to 7 classes.
Using 2003 files for validation.


In [6]:
# Normalize the datasets
train_dataset = train_dataset.map(lambda x,y: (layers.Rescaling(1./255)(x), y))
validation_dataset = validation_dataset.map(lambda x,y: (layers.Rescaling(1./255)(x), y))


In [7]:
x_test, y_test, labels_test = load_images("Data_Cancer_Test/")
x_test = x_test/255.0

### VGG Model

In [10]:
model_vgg = Sequential([
    keras.layers.Input((224,224,3)),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Flatten(),
    
    Dense(4096),
    Dense(4096),
    Dense(1000),
    
    Dense(7, activation = "softmax")
    
])

In [11]:
model_vgg.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,00

 Total params: 143,674,247 (548.07 MB)

 Trainable params: 143,674,247 (548.07 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
epochs = 10
model_vgg.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

history_vgg = model_vgg.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

Epoch 1/10


I0000 00:00:1720863607.046340     846 service.cc:145] XLA service 0x7ff0a0025ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720863607.047583     846 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-07-13 11:40:07.356897: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-13 11:40:08.206889: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720863609.116317    1076 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 196 bytes spill stores, 196 bytes spill loads

I0000 00:00:1720863609.426768    1074 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 132 bytes spill stores, 132 bytes spill loads

I0000 

125/126 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.6089 - loss: 1.4943

I0000 00:00:1720863752.801012    1237 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1510', 88 bytes spill stores, 88 bytes spill loads

I0000 00:00:1720863752.888345    1236 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1499', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1720863753.640221    1241 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1521', 668 bytes spill stores, 616 bytes spill loads



126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 739ms/step - accuracy: 0.6093 - loss: 1.4922

I0000 00:00:1720863779.272303    1396 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_228', 232 bytes spill stores, 208 bytes spill loads

I0000 00:00:1720863779.379035    1398 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_228', 272 bytes spill stores, 272 bytes spill loads

I0000 00:00:1720863779.578878    1394 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_224', 224 bytes spill stores, 200 bytes spill loads

I0000 00:00:1720863779.850914    1401 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_232', 720 bytes spill stores, 636 bytes spill loads

I0000 00:00:1720863780.234757    1393 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_232', 3644 bytes spill stores, 3636 bytes spill loads

I0000 00:00:1720863780.

126/126 ━━━━━━━━━━━━━━━━━━━━ 183s 866ms/step - accuracy: 0.6097 - loss: 1.4902 - val_accuracy: 0.6670 - val_loss: 1.2139
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 79s 626ms/step - accuracy: 0.6708 - loss: 1.1508 - val_accuracy: 0.6670 - val_loss: 1.1475
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 79s 623ms/step - accuracy: 0.6701 - loss: 1.1480 - val_accuracy: 0.6670 - val_loss: 1.1502
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 81s 641ms/step - accuracy: 0.6641 - loss: 1.1623 - val_accuracy: 0.6670 - val_loss: 1.1843
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 79s 628ms/step - accuracy: 0.6686 - loss: 1.1492 - val_accuracy: 0.6670 - val_loss: 1.2052
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 82s 646ms/step - accuracy: 0.6640 - loss: 1.1673 - val_accuracy: 0.6670 - val_loss: 1.1695
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 80s 635ms/step - accuracy: 0.6660 - loss: 1.1500 - val_accuracy: 0.6670 - val_loss: 1.1567
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 80s 630ms/step - accuracy: 0.6652 - loss: 1.1549 - va

In [14]:
model_vgg.save("Models/vgg_19_model.keras")

In [ ]:
model_vgg = keras.saving.load_model("Models/vgg_19_model.keras")

In [ ]:
predictions = model.predict(x_test)

### 34-Layer Plain Neural Network

In [ ]:
model_34 = Sequential([
    Input((img_height, img_weight))
    Conv2D(64, (7,7), strides = 2, padding = "same", activation = "relu"),
    MaxPooling2D((2,2)),
    
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(64, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(128, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(128, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(256, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    Conv2D(512, (3,3), strides = 2, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    Conv2D(512, (3,3), strides = 1, padding = "same", activation = "relu"),
    
    GlobalAveragePooling2D(),
    Dense(1000),
    Dense(7, activation = "softmax")
    
    
])

In [13]:
keras.backend.clear_session()